In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("football.csv")
print(data.dtypes)
print(data.shape)
data.head()

date          object
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object
(41586, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [3]:
# Change date to year
data['date'] = pd.to_datetime(data['date']).dt.year
print(data.dtypes)
data.head()

date           int64
home_team     object
away_team     object
home_score     int64
away_score     int64
tournament    object
city          object
country       object
neutral         bool
dtype: object


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873,England,Scotland,4,2,Friendly,London,England,False
2,1874,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875,England,Scotland,2,2,Friendly,London,England,False
4,1876,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


# Number of Games

In [4]:
# Group by date and rename
yearlyGames = data.copy()
yearlyGames = yearlyGames.groupby(['date'], as_index=False).count()
yearlyGames = yearlyGames[['date','home_team']]
yearlyGames = yearlyGames.rename(columns={"date":"year","home_team":"count"})
yearlyGames.head()

,year,count
0,1872,1
1,1873,1
2,1874,1
3,1875,1
4,1876,2


In [5]:
# Export to csv
yearlyGames.to_csv("football_q1.csv", index=False)

# Win Percentage

In [6]:
# Filter for games from 1970
winner = data.copy()
winner = winner[winner['date']>=1970] 
print(winner.shape)
winner.head()

(34238, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
7348,1970,Malta,Luxembourg,1,1,Friendly,Gżira,Malta,False
7349,1970,England,Netherlands,0,0,Friendly,London,England,False
7350,1970,Israel,Netherlands,0,1,Friendly,Jaffa,Israel,False
7351,1970,Peru,Czechoslovakia,0,2,Friendly,Lima,Peru,False
7352,1970,Cameroon,Ivory Coast,3,2,African Cup of Nations,Khartoum,Sudan,True


In [7]:
# Find winner of games
winner['winner'] = np.where(winner['home_score']>winner['away_score'],winner['home_team'],None)
winner['winner'] = np.where(winner['home_score']<winner['away_score'],winner['away_team'],winner['winner'])
winner.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
7348,1970,Malta,Luxembourg,1,1,Friendly,Gżira,Malta,False,None
7349,1970,England,Netherlands,0,0,Friendly,London,England,False,None
7350,1970,Israel,Netherlands,0,1,Friendly,Jaffa,Israel,False,Netherlands
7351,1970,Peru,Czechoslovakia,0,2,Friendly,Lima,Peru,False,Czechoslovakia
7352,1970,Cameroon,Ivory Coast,3,2,African Cup of Nations,Khartoum,Sudan,True,Cameroon


In [8]:
# Count number of games played
win_pct = pd.DataFrame(winner['away_team'].value_counts().add(winner['home_team'].value_counts(), fill_value=0), 
                       columns=["played"])
# Count number of games won
win_pct['won'] = winner['winner'].value_counts()
# Fill NaN in games won with 0, change dtype
win_pct['won']= win_pct['won'].fillna(0)
win_pct = win_pct.astype({'played': 'int64','won': 'int64'})
print(win_pct.shape)
win_pct.head()

(308, 2)


,played,won
Abkhazia,28,12
Afghanistan,109,30
Albania,297,76
Alderney,19,3
Algeria,480,206


In [9]:
# Get code for countries
world_code = pd.read_csv("world_code.csv")
world_code = world_code.set_index("country")
print(world_code.shape)
world_code.head()

(246, 1)


,code
country,
Antigua and Barbuda,ATG
Algeria,DZA
Azerbaijan,AZE
Albania,ALB
Armenia,ARM


In [10]:
# Join games stats with code
win_pct = win_pct.join(world_code)
win_pct = win_pct.reset_index()
win_pct = win_pct.rename(columns={"index":"country"})
win_pct.head()

,country,played,won,code
0,Abkhazia,28,12,NaN
1,Afghanistan,109,30,AFG
2,Albania,297,76,ALB
3,Alderney,19,3,NaN
4,Algeria,480,206,DZA


In [11]:
# Calculate win percentage
win_pct['win_pct'] = win_pct['won']/win_pct['played']
print(win_pct.shape)
win_pct.head()

(308, 5)


,country,played,won,code,win_pct
0,Abkhazia,28,12,NaN,0.428571
1,Afghanistan,109,30,AFG,0.275229
2,Albania,297,76,ALB,0.255892
3,Alderney,19,3,NaN,0.157895
4,Algeria,480,206,DZA,0.429167


In [12]:
# Filter top 25% of teams in terms of games played
top_quartile = win_pct['played'].quantile(0.75)
print(top_quartile)
win_pct = win_pct[win_pct['played']>top_quartile]
win_pct = win_pct.dropna()
win_pct.head()

377.25


,country,played,won,code,win_pct
4,Algeria,480,206,DZA,0.429167
12,Argentina,579,301,ARG,0.519862
16,Australia,444,221,AUS,0.497748
17,Austria,425,167,AUT,0.392941
20,Bahrain,485,164,BHR,0.338144


In [13]:
win_pct = win_pct.sort_values(by="win_pct", ascending=False)
win_pct = win_pct.reset_index(drop=True)
win_pct.head(10)

,country,played,won,code,win_pct
0,Brazil,669,434,BRA,0.648729
1,Spain,521,319,ESP,0.612284
2,Germany,617,377,DEU,0.611021
3,France,528,306,FRA,0.579545
4,Netherlands,498,283,NLD,0.568273
5,England,564,308,GBR,0.546099
6,Iran,478,260,IRN,0.543933
7,Portugal,460,245,PRT,0.532609
8,Italy,539,283,ITA,0.525046
9,South Korea,761,396,KOR,0.520368


In [14]:
win_pct['rank'] = win_pct.index + 1
win_pct.head(10).to_csv("football_q2.csv", index=False)

# World Cup

In [15]:
# Filter for past 2 World Cups
wc = data.copy()
wc = wc[wc['tournament'].isin(["FIFA World Cup"])] #,"FIFA World Cup qualification"])]
wc = wc[wc['date']>=2014]
print(wc.shape)
wc.head()

(128, 9)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
36088,2014,Brazil,Croatia,3,1,FIFA World Cup,São Paulo,Brazil,False
36089,2014,Chile,Australia,3,1,FIFA World Cup,Cuiabá,Brazil,True
36090,2014,Mexico,Cameroon,1,0,FIFA World Cup,Natal,Brazil,True
36091,2014,Spain,Netherlands,1,5,FIFA World Cup,Salvador,Brazil,True
36092,2014,Colombia,Greece,3,0,FIFA World Cup,Belo Horizonte,Brazil,True


In [16]:
# Get game winners
wc['winner'] = np.where(wc['home_score']>wc['away_score'],wc['home_team'],None)
wc['winner'] = np.where(wc['home_score']<wc['away_score'],wc['away_team'],wc['winner'])
wc.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
36088,2014,Brazil,Croatia,3,1,FIFA World Cup,São Paulo,Brazil,False,Brazil
36089,2014,Chile,Australia,3,1,FIFA World Cup,Cuiabá,Brazil,True,Chile
36090,2014,Mexico,Cameroon,1,0,FIFA World Cup,Natal,Brazil,True,Mexico
36091,2014,Spain,Netherlands,1,5,FIFA World Cup,Salvador,Brazil,True,Netherlands
36092,2014,Colombia,Greece,3,0,FIFA World Cup,Belo Horizonte,Brazil,True,Colombia


In [17]:
# Count number of wins by country
world_cup = pd.DataFrame(wc['winner'].value_counts())
world_cup.head()

,winner
Belgium,10
France,9
Germany,7
Brazil,6
Uruguay,6


In [18]:
# Count goal difference for home country
home_score = wc.groupby('home_team').sum()
home_score = home_score['home_score'] - home_score['away_score']
home_score.head()

home_team
Algeria                    0
Argentina                  1
Australia                 -6
Belgium                   12
Bosnia and Herzegovina     2
dtype: int64

In [19]:
# Count goal difference for away country
away_score = wc.groupby('away_team').sum()
away_score = away_score['away_score'] - away_score['home_score']
away_score.head()

away_team
Algeria                   0
Argentina                 0
Australia                -3
Belgium                   1
Bosnia and Herzegovina   -2
dtype: int64

In [20]:
# Calculate overall goal difference
world_cup['goal_diff'] = (home_score+away_score)
world_cup = world_cup.rename(columns={"winner":"won"})
world_cup.head()

,won,goal_diff
Belgium,10,13
France,9,15
Germany,7,12
Brazil,6,2
Uruguay,6,2


In [21]:
# Normalize values using min-max
max_won = world_cup['won'].max()
min_won = world_cup['won'].min()
world_cup['won_norm'] = (world_cup['won']-min_won)/(max_won-min_won)
world_cup.head()

,won,goal_diff,won_norm
Belgium,10,13,1.000000
France,9,15,0.888889
Germany,7,12,0.666667
Brazil,6,2,0.555556
Uruguay,6,2,0.555556


In [22]:
# Normalize values using min-max
max_diff = world_cup['goal_diff'].max()
min_diff = world_cup['goal_diff'].min()
world_cup['goal_diff_norm'] = (world_cup['goal_diff']-min_diff)/(max_diff-min_diff)
world_cup.head()

,won,goal_diff,won_norm,goal_diff_norm
Belgium,10,13,1.000000,0.90
France,9,15,0.888889,1.00
Germany,7,12,0.666667,0.85
Brazil,6,2,0.555556,0.35
Uruguay,6,2,0.555556,0.35


In [23]:
#Calculate score
world_cup['score'] = 50*(world_cup['goal_diff_norm']+world_cup['won_norm'])
world_cup = world_cup.sort_values(by="score", ascending=False)
world_cup = world_cup.reset_index()
world_cup = world_cup.rename(columns={"index":"country"})
world_cup['rank'] = world_cup.index+1
world_cup.head(10)

,country,won,goal_diff,won_norm,goal_diff_norm,score,rank
0,Belgium,10,13,1.000000,0.90,95.000000,1
1,France,9,15,0.888889,1.00,94.444444,2
2,Germany,7,12,0.666667,0.85,75.833333,3
3,Colombia,6,11,0.555556,0.80,67.777778,4
4,Netherlands,5,11,0.444444,0.80,62.222222,5
5,Croatia,5,5,0.444444,0.50,47.222222,6
6,Brazil,6,2,0.555556,0.35,45.277778,7
7,Uruguay,6,2,0.555556,0.35,45.277778,8
8,Argentina,6,1,0.555556,0.30,42.777778,9
9,England,3,2,0.222222,0.35,28.611111,10


In [24]:
world_cup.to_csv("football_q3.csv", index=False)